In [1]:
import pandas as pd

In [2]:
filename = 'query_all_org_class_results.xlsx'
outfile = 'output_class_to_orgs_20190429_v7.xlsx'

filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/all_org_classes'
read_file_from = '{0}/{1}'.format(filepath, filename)
write_file_to = '{0}/{1}'.format(filepath, outfile)

In [3]:
df = pd.read_excel(read_file_from)
df = df.fillna(0)
df.head()

,graph,orgName,class,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,chrj_pr,Organization,大興紡織,0,0,0,0,0,0,0,0
1,chrj_pr,Organization,裕華紡織廠,0,0,0,0,0,0,0,0
2,chrj_pr,Organization,大華紡織廠,0,0,0,0,0,0,0,0
3,chrj_pr,Organization,五月,0,0,0,0,0,0,0,0
4,chrj_pr,EducationalOrganization,京都大學紡織,0,0,0,0,0,0,0,0


In [4]:
print(df.count())
count = max(df.count())
print('max count = {0}'.format(count))

graph          35556
orgName        35556
class          35556
Unnamed: 3     35556
Unnamed: 4     35556
Unnamed: 5     35556
Unnamed: 6     35556
Unnamed: 7     35556
Unnamed: 8     35556
Unnamed: 9     35556
Unnamed: 10    35556
dtype: int64
max count = 35556


In [5]:
count = df.groupby(['orgName']).count()['graph']

count = count.sort_values(ascending=False)
print(count)

maxCount = max(count)

orgName
Organization                                 8597
PrivateOrganization                          7834
OrganizationUnit                             4773
PublicOrganization                           3759
StateOrganization                            3753
ElectedOrganization                          1436
FinancialOrganization                        1175
AdministrativeOrganization                    904
HealthOrganization                            683
AgriculturalOrganization                      634
EducationalOrganization                       435
ProvincialOrganization                        286
MonopolyOrganization                          254
EarlyPostWarOrganization                      207
Trade_FinancialOrganization                   168
ProfessionalOrganization                      154
ReligiousOrganization                          87
CommunityOrganization                          68
ForestryOrganization                           65
PartyOrganization                         

In [6]:
results = []

for idx in range(0, maxCount):
    row = df.loc[idx]
#     print(row)
    graph = row['graph']
    orgName = row['orgName']
    className = row['class']
    
    comments = ''
    for cidx in range(3, len(row)):
        if row[cidx] != 0:
            comments += '{0};'.format(row[cidx])
    
    result = {'graph': graph, 'org': orgName, 'class': className, 'comments': comments}
    results.append(result)
    
#     print(graph, orgName, className, comments)
results[0:5]

[{'graph': 'chrj_pr', 'org': 'Organization', 'class': '大興紡織', 'comments': ''},
 {'graph': 'chrj_pr', 'org': 'Organization', 'class': '裕華紡織廠', 'comments': ''},
 {'graph': 'chrj_pr', 'org': 'Organization', 'class': '大華紡織廠', 'comments': ''},
 {'graph': 'chrj_pr', 'org': 'Organization', 'class': '五月', 'comments': ''},
 {'graph': 'chrj_pr',
  'org': 'EducationalOrganization',
  'class': '京都大學紡織',
  'comments': ''}]

In [7]:
classResults = {}

for res in results:
    className = res['class']
    graph = res['graph']
    orgName = res['org']
    comments = res['comments']

    if className not in classResults:
        classResults[className] = {'orgGra': {}, 'comments': ''}
        
    orgGraph = classResults[className]['orgGra']
    if orgName not in orgGraph:
        orgGraph[orgName] = []
    
    if graph not in orgGraph[orgName]:
        orgGraph[orgName].append(graph)
#     print(className, orgName, graph, comments)
    
    classResults[className] = {'orgGra': orgGraph, 'comments': comments}
#     if comments != '':
#         print(classResults[className])

# {'orgGra': {'PrivateOrganization': ['yjc']}, 'comments': '栽培南洋種鳳梨;鳳梨罐頭銷遍全島;'}
# for org, orgGraph in classResults.items():
#     if orgGraph['comments'] != '':
#         print(org, orgGraph)

# classResults

In [8]:
def mapFunc(orgName, defaultVal, mapTable):
    res = defaultVal;
    for tKey in mapTable:
        for tVale in mapTable[tKey]:
            if tVale in orgName:
                return tKey

    return res

## Ownership mapping table

In [9]:
# Ownership mapping table
OwnershipMapping = {
    'PrivateOrganization': ['PrivateOrganization'], 
    'StateOrganization': ['StateOrganization'],
    'PublicOrganization': ['PublicOrganization']
}
UnknownOwnership = 'UnknownOwnership';

## ModeOfEntry mapping table

In [10]:
# ModeOfEntry mapping table
ModeOfEntryMapping = {
    'Appointed': ['總督府','街','庄','州','公學校','利組合','組合'], 
    'Elected': ['協議會']
}
UnknownModeOfEntry = 'UnknownModeOfEntry';

## Period mapping table

In [11]:
## Period mapping table
# 總督府，街，庄，信託會社，州，公學校，株式會，株式会社，利組合，組合，國語講習所
PeriodMapping = {
    'JapaneseColonialOrganization': ['總督府','街','庄','信託會社','州','公學校',
                                     '株式會','株式会社','利組合','組合','國語講習所'],
    'EarlyPostWarOrganization': ['縣議會','台灣省','台灣長官公署'],
    'RepublicanOrganization': [],
}
UnknownPeriod = 'UnknownPeriodOrganization';

## LevelOfOperation mapping table

In [12]:
## LevelOfOperation mapping table
LevelOfOperationMapping = {
    'CentralOrganization': [],
    'LocalOrganization': ['庄','街','區','州','市','縣'],
    'ProvincialOrganization': ['省','廳']
}
UnknownLevelOfOperation = 'UnknwownLevelOfOperation';

## Function mapping table

In [13]:
## Function mapping table
func_filename = 'query_org_divisionContent_results.csv'
func_file_from = '{0}/{1}'.format(filepath, func_filename)

funcDf = pd.read_csv(func_file_from)
funcDf.head()

def mapOrgToFunction(orgName, defaultVal):
#     print(orgName)
    row = funcDf.loc[funcDf['sName'] == orgName]
    if row.empty:
        division = [defaultVal]
    else:
        division = row['orgName'].tolist()

    return division

UnknownFunction = 'UnknwownFunction';
# print(mapOrgToFunction('壯圍庄古亭笨第十五保第七保保甲', UnknownFunction))
# print(mapOrgToFunction('高雄州農會', UnknownFunction))
# print(mapOrgToFunction('在魚池庄經營製茶業', UnknownFunction))
# print(mapOrgToFunction('xxxxx', UnknownFunction))

# for idx in range(0, len(funcDf.index)):
# # for idx in range(16,23):
# #     print(funcDf.loc[idx, 'sName'])
#     print(mapOrgToFunction(funcDf.loc[idx, 'sName'], UnknownFunction))


## Main generation

In [14]:
orgList = []
for className, orgGraph in classResults.items():
    strClassName = str(className)
    for org in orgGraph['orgGra']:
        orgArr = []
#         graphs = ';'.join(orgGraph['orgGra'][org])
        
#         orgStr = '{0}/({1})'.format(org, graphs)
#         orgArr.append(orgStr)
        # Organization
        orgArr.append(strClassName)
        
        # Ownership
        orgArr.append(mapFunc(org, UnknownOwnership, OwnershipMapping))
        
        # ModeOfEntry
        orgArr.append(mapFunc(strClassName, UnknownModeOfEntry, ModeOfEntryMapping))
    
        # Period
        orgArr.append(mapFunc(strClassName, UnknownPeriod, PeriodMapping))
        
        # LevelOfOperation
        orgArr.append(mapFunc(strClassName, UnknownLevelOfOperation, LevelOfOperationMapping))
        
        # Function
        funcArray = mapOrgToFunction(strClassName, UnknownFunction)
#         orgArr.append(mapOrgToFunction(strClassName, UnknownFunction))

        # comments
        comments = orgGraph['comments']
        
        for funcName in funcArray:
            finalOrg = []
            finalOrg = orgArr.copy()
            finalOrg.append(funcName)
            finalOrg.append(comments)
            
#             if strClassName == '在魚池庄經營製茶業':
#                 print(orgArr)
#                 print(finalOrg)
            if finalOrg not in orgList:
                orgList.append(finalOrg)
    
orgList[0:5]
#         if len(orgGraph['orgGra'][org]) > 1:
# #             print(className, orgGraph)
#             print(className, org, graphs)
#         print(org, graphs)
#     if orgGraph['comments'] != '':
#         print(org, orgGraph)

[['大興紡織',
  'UnknownOwnership',
  'UnknownModeOfEntry',
  'UnknownPeriodOrganization',
  'UnknwownLevelOfOperation',
  'UnknwownFunction',
  ''],
 ['裕華紡織廠',
  'UnknownOwnership',
  'UnknownModeOfEntry',
  'UnknownPeriodOrganization',
  'UnknwownLevelOfOperation',
  'UnknwownFunction',
  ''],
 ['大華紡織廠',
  'UnknownOwnership',
  'UnknownModeOfEntry',
  'UnknownPeriodOrganization',
  'UnknwownLevelOfOperation',
  'UnknwownFunction',
  ''],
 ['五月',
  'UnknownOwnership',
  'UnknownModeOfEntry',
  'UnknownPeriodOrganization',
  'UnknwownLevelOfOperation',
  'UnknwownFunction',
  ''],
 ['京都大學紡織',
  'UnknownOwnership',
  'UnknownModeOfEntry',
  'UnknownPeriodOrganization',
  'UnknwownLevelOfOperation',
  'UnknwownFunction',
  '']]

In [15]:
def getMaxLen(orgList):
    maxEleLen = 0
    for org in orgList:
        orgLen = len(org)
        if maxEleLen < orgLen:
            maxEleLen = orgLen
    return maxEleLen

print(getMaxLen(orgList))
columns = ['Organization','Ownership','ModeOfEntry','Period','LevelOfOperation','Function','comments']    
columns

7


['Organization',
 'Ownership',
 'ModeOfEntry',
 'Period',
 'LevelOfOperation',
 'Function',
 'comments']

In [16]:
orgDf = pd.DataFrame(orgList, columns=columns)
# orgDf = orgDf.fillna('')
orgDf.head()

,Organization,Ownership,ModeOfEntry,Period,LevelOfOperation,Function,comments
0,大興紡織,UnknownOwnership,UnknownModeOfEntry,UnknownPeriodOrganization,UnknwownLevelOfOperation,UnknwownFunction,
1,裕華紡織廠,UnknownOwnership,UnknownModeOfEntry,UnknownPeriodOrganization,UnknwownLevelOfOperation,UnknwownFunction,
2,大華紡織廠,UnknownOwnership,UnknownModeOfEntry,UnknownPeriodOrganization,UnknwownLevelOfOperation,UnknwownFunction,
3,五月,UnknownOwnership,UnknownModeOfEntry,UnknownPeriodOrganization,UnknwownLevelOfOperation,UnknwownFunction,
4,京都大學紡織,UnknownOwnership,UnknownModeOfEntry,UnknownPeriodOrganization,UnknwownLevelOfOperation,UnknwownFunction,


In [17]:
orgDf.to_excel(write_file_to)